In [111]:
import gurobipy as gp

In [122]:
import numpy as np
import numpy.random as npr
seed = 1
rng = npr.default_rng(seed)
N = 100 #25 50 75 100
v = np.zeros(N)
v = rng.uniform(0.5, 1.5, N)
# for i in range(N):
#    utility = rng.integers(2)*i*0.05 + rng.random()*0.1
#    v[i] = max(round(utility, 2), 0.01) 
print(v)

[1.01182162 1.4504637  0.64415961 1.44864945 0.81183145 0.92332645
 1.32770259 0.90919914 1.04959369 0.52755911 1.25351311 1.03814331
 0.82973172 1.2884287  0.80319483 0.95349789 0.6340417  0.90311299
 0.70345524 0.76231334 1.25036467 0.78040876 0.98519097 1.4807372
 1.46165719 1.22478994 1.04122686 0.7768912  0.66065201 1.46992541
 1.01606859 0.61586561 1.12348976 1.27668311 1.1130033  1.4172977
 0.53959288 1.02858926 0.95933588 0.56234958 1.14132817 1.35263284
 1.09294102 0.76009745 1.33988152 1.00949588 1.01088888 1.25303021
 0.64792204 1.31962672 1.18328691 1.28709694 0.69161626 1.30236416
 0.69132393 0.58155262 1.35522697 1.3612835  1.3765371  0.97190972
 0.77404839 0.50709183 1.1457209  1.21990938 1.33556922 0.78187783
 0.71521817 1.13933138 1.30505483 1.46367087 0.65052483 0.98221239
 1.39471586 0.92271691 1.08950206 0.52449068 1.17345989 1.41908862
 1.32682533 1.38552027 1.16035538 0.74555227 1.268517   0.71167474
 1.33127483 0.56271792 1.32548781 0.66450727 0.875147   0.816738

In [113]:
def profit_mnl(p, S):
    global v
    pi = 0
    Vj_S = 0
    num = 0
    for i in S:
        num += p[i]*v[i]
        Vj_S += v[i]
    pi += num/(1+Vj_S)
    return pi

In [114]:
def nested_by_price_mnl(p):
    global N
    profits = [0 for i in range(N)]
    S = []
    for i in range(N):
        S.append(i)
        profits[i] = profit_mnl(p, S)
    best_S = [i for i in range(np.argmax(profits)+1)]
    return max(profits), best_S

In [115]:
def find_best_option_mnl(p, S):
    global v, N
    current_best = profit_mnl(p, S)
    bestS = S
    flag = 0
    N_S = [k for k in range(N) if k not in S]
    for k in N_S:
        newS = S + [k]
        if profit_mnl(p, newS) >= current_best:
            flag = 1
            bestS = newS
            current_best = profit_mnl(p, newS)
    return current_best, bestS, flag

def greedy_mnl(p):
    global v, N
    max_profit = -1
    best_S = []
    S = []
    flag = 1
    while flag==1:
        max_profit, S, flag = find_best_option_mnl(p, S)
    best_S = sorted(S)
    return max_profit, best_S

In [116]:
import matplotlib.pyplot as plt
import math
import statistics as st
import time

In [117]:
n_simulations = 100
avg_price = 12
seed = 1
rng = npr.default_rng(seed)
#compute prices
psim = [0 for k in range(n_simulations)]
for k in range(n_simulations):
    psim[k] = abs(rng.normal(avg_price, avg_price, N))
    # psim[k] = rng.uniform(avg_price - math.sqrt(3*avg_price), avg_price + math.sqrt(3*avg_price), N)
    # psim[k] = rng.exponential(avg_price, N)
    psim[k] = np.round(psim[k], 2)
    psim[k] = sorted(psim[k], reverse=True)

In [118]:
# problem solved using the nested formulation
profits_nested = []
times_nested = []
bestS_nested = []

for k in range(n_simulations):

    start_time = time.time()
    profit, bestS = nested_by_price_mnl(psim[k])
    end_time = time.time()

    profits_nested.append(profit)
    times_nested.append(end_time-start_time)
    bestS_nested.append(bestS)
    
avg_time_nested = st.mean(times_nested)

In [119]:
# problem solved using the greedy formulation
profits_greedy = []
times_greedy = []
bestS_greedy = []

for k in range(n_simulations):
    
    start_time = time.time()
    profit, bestS = greedy_mnl(psim[k])
    end_time = time.time()
    
    profits_greedy.append(profit)
    times_greedy.append(end_time-start_time)
    bestS_greedy.append(bestS)

avg_time_greedy = st.mean(times_greedy)

In [120]:
# counting how many instances are different
count = 0
different_results_greedy = []

for k in range(n_simulations):
    if bestS_nested[k] != bestS_greedy[k]:
        count += 100/n_simulations
        different_results_greedy.append(100*profits_greedy[k]/profits_nested[k])
        # print(bestS_nested[k], bestS_greedy[k], profits_nested[k], profits_greedy[k])


In [121]:
if different_results_greedy != []:
    print(' & ' + str(count) + ' & ' + str(round(min(different_results_greedy), 2)) \
          + ' & ' + str(round(st.mean(different_results_greedy),2)) \
            + ' & ' + str(round(100*avg_time_greedy/avg_time_nested, 2)))
else:
    print(' & 0 & NA & NA & ' + str(round(100*avg_time_greedy/avg_time_nested, 2)))

 & 0 & NA & NA & 96.08
